In [3]:
import json 
import pandas as pd 
import os
file_path = '/mnt/d/dataStorage/UPNepseDataLake/historicFinancialData/GLBSL_financial_data'
os.listdir(file_path)

['corporate_action_file.json',
 'dividend_file.json',
 'financials_file.json',
 'growth_file.json',
 'key_ratio_file.json']

In [25]:
files = os.listdir(file_path)
for file_name in files:
    filepath = os.path.join(file_path,f"{file_name}")
    data_dict = {}
    with open(filepath,'r') as js_file:
        data_dict = json.load(js_file)
        for k,v in data_dict.items():
            print(k)

corporate_action
dividend
financials
growth
key_ratio


In [29]:
for k,v in data_dict.items():
    for ke,va in v.items():
        print(va)

{'stock': {'pe_ratio': '-76.60952280513', 'ev_ebit': 19.586749739789592, 'pb_ratio': '5.4363470652263', 'peg_ratio': None, 'price_cash_flow': -24.556888615644258, 'price_to_sale': '12.209197475203', 'dividend_yield': 0, 'earning_yield': -0.013053207530657458, 'earning_yield_ev_ebit': 0.05105492301096518, 'pe_ratio_5_yrs': 56.64286625426447}, 'sectors': {'median': {'pe_ratio': 58.4341822151615, 'ev_ebit': 17.70089928697468, 'pb_ratio': '4.4748372190391', 'peg_ratio': 2.8930233030683503, 'price_cash_flow': 15.416819830264586, 'price_to_sale': '4.3046565700999', 'dividend_yield': 1.8181818181818001, 'earning_yield': 0.01713343808484928, 'earning_yield_ev_ebit': 0.05649430482528402, 'pe_ratio_5_yrs': 36.31791214865092}, 'median_symbol': {'pe_ratio': '', 'ev_ebit': '', 'pb_ratio': '', 'peg_ratio': '', 'price_cash_flow': '', 'price_to_sale': '', 'dividend_yield': '', 'earning_yield': '', 'earning_yield_ev_ebit': '', 'pe_ratio_5_yrs': ''}, 'high': {'pe_ratio': '693.25024926331', 'ev_ebit': 31

In [31]:

stock_data = pd.read_csv('https://query1.finance.yahoo.com/v7/finance/download/AAPL?period1=0&period2=9999999999&interval=1d&events=history')

In [32]:
stock_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099192,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.094017,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.087117,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089273,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091861,73449600
...,...,...,...,...,...,...,...
10897,2024-03-06,171.059998,171.240005,168.679993,169.119995,169.119995,68587700
10898,2024-03-07,169.149994,170.729996,168.490005,169.000000,169.000000,71765100
10899,2024-03-08,169.000000,173.699997,168.940002,170.729996,170.729996,76114600
10900,2024-03-11,172.940002,174.380005,172.050003,172.750000,172.750000,60139500


In [35]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Load the stock dataset (you can replace this with your own dataset)
stock_data = pd.read_csv('https://query1.finance.yahoo.com/v7/finance/download/AAPL?period1=0&period2=9999999999&interval=1d&events=history')

# Convert the 'Date' column to datetime format
stock_data['Date'] = pd.to_datetime(stock_data['Date'])

# Define the time window size and overlap
window_size = 30  # Number of days per window
overlap = 15      # Number of days to overlap between windows

# Group data into time windows
time_windows = []
start_date = stock_data['Date'].min()
end_date = stock_data['Date'].max()
while start_date <= end_date:
    end_window = start_date + pd.Timedelta(days=window_size)
    if end_window > end_date:
        break
    time_window_data = stock_data[(stock_data['Date'] >= start_date) & (stock_data['Date'] < end_window)]
    time_windows.append(time_window_data)
    start_date += pd.Timedelta(days=window_size - overlap)

# Define a function to extract features and perform clustering for each time window
def cluster_time_windows(time_windows):
    cluster_traces = []
    for i, window_data in enumerate(time_windows):
        print(f"Clustering Time Window {i+1}/{len(time_windows)}...")
        
        # Skip clustering if there are not enough data points
        if len(window_data) < 2:
            print("Skipping clustering due to insufficient data points.")
            continue
        
        # Extract features from the time window data
        features = {}
        features['slope'] = (window_data['Close'].iloc[-1] - window_data['Close'].iloc[0]) / len(window_data)
        features['std'] = window_data['Close'].std()
        features['percentage_change'] = (window_data['Close'].iloc[-1] - window_data['Close'].iloc[0]) / window_data['Close'].iloc[0] * 100
        features['range'] = window_data['Close'].max() - window_data['Close'].min()
        
        # Normalize features
        scaler = StandardScaler()
        normalized_features = scaler.fit_transform(np.array(list(features.values())).reshape(1, -1))
        
        # Apply clustering algorithm (e.g., k-means)
        n_clusters = 3  # Number of clusters
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        cluster_labels = kmeans.fit_predict(normalized_features)
        
        # Visualize the clusters
        cluster_data = window_data.copy()
        cluster_data['Cluster'] = cluster_labels
        cluster_traces.append(go.Scatter(
            x=cluster_data['Date'],
            y=cluster_data['Close'],
            mode='lines',
            name=f"Window {i+1}, Cluster {cluster_labels[0]}",
            line=dict(width=2)
        ))
        
    # Create layout
    layout = go.Layout(
        title='Stock Price Clusters Over Time',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Stock Price')
    )
    
    # Create figure
    fig = go.Figure(data=cluster_traces, layout=layout)
    
    # Plot the figure
    fig.show()

# Cluster time windows and visualize clusters
cluster_time_windows(time_windows)

Clustering Time Window 1/1052...


ValueError: n_samples=1 should be >= n_clusters=3.